In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
import matplotlib.pyplot as plt
from branca.element import Template, MacroElement
import folium

In [2]:
# Template para legenda
template = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>jQuery UI Draggable - Default functionality</title>
  <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

  <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
  <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>
  
  <script>
  $( function() {
    $( "#maplegend" ).draggable({
                    start: function (event, ui) {
                        $(this).css({
                            right: "auto",
                            top: "auto",
                            bottom: "auto"
                        });
                    }
                });
});

  </script>
</head>
<body>

 
<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:14px; right: 20px; bottom: 20px;'>
     
<div class='legend-title'>Dependencia Escola</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:green;opacity:0.7;'></span>Federal</li>
    <li><span style='background:blue;opacity:0.7;'></span>Municipal</li>
    <li><span style='background:red;opacity:0.7;'></span>Estadual</li>
    <li><span style='background:purple;opacity:0.7;'></span>Privada</li>

  </ul>
</div>
</div>
 
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""

In [3]:
# Ler Arquivos
df = pd.read_csv('11_Escolas_Coordenadas.csv', sep = ';', encoding='latin-1', low_memory = False)

df_mun = pd.read_csv('escolas122020.csv', sep = ';', encoding='latin-1', low_memory = False)

df_censo = pd.read_csv('microdados_censo_escolar_2020/microdados_educacao_basica_2020/DADOS/escolas.CSV', sep = '|', encoding='latin-1', low_memory = False)

distritos = gpd.read_file('DISTRITO_MUNICIPAL_SP_SMDU/DISTRITO_MUNICIPAL_SP_SMDUPolygon.shp')

In [4]:
# Preparação dos dados
df_sp = df[df['MUN']=='SAO PAULO']

df_sp.loc[:,'COD_ESC']= df_sp.loc[:,'COD_ESC']+35000000

df_censo_SP = df_censo[df_censo['CO_MUNICIPIO']==3550308]

df_sp_select = df_sp.loc[:,['COD_ESC', 'DS_LATITUDE', 'DS_LONGITUDE']]
df_sp_select.columns = ['codesc', 'LAT', 'LNG']
df_sp_select['LAT'] = pd.to_numeric(df_sp_select['LAT'].str.replace(',','.'))
df_sp_select['LNG'] = pd.to_numeric(df_sp_select['LNG'].str.replace(',','.'))

df_mun_select = df_mun.loc[:,['codinep','LATITUDE', 'LONGITUDE']]
df_mun_select.columns = ['codesc', 'LAT', 'LNG']
df_mun_select['LAT'] = df_mun_select['LAT']/1000000
df_mun_select['LNG'] = df_mun_select['LNG']/1000000


df_lat_lng = pd.concat([df_sp_select,df_mun_select], ignore_index=True)

df_lat_lng.dropna(inplace = True)
df_lat_lng.drop(df_lat_lng[df_lat_lng['LNG']<=-46.9].index,inplace = True)
df_lat_lng.loc[:,'codesc'] = df_lat_lng.loc[:,'codesc'].astype(int)


df_etapas = pd.read_csv('etapas.csv')
df_etapas.loc[:,'cod_etapa'] = df_etapas.loc[:,'cod_etapa'].astype(int)
dict_dependencia = {1: "Federal",2 :"Estadual", 3 :"Municipal",4:"Privada"}

df_turma = pd.read_csv('microdados_censo_escolar_2020/microdados_educacao_basica_2020/DADOS/turmas.CSV', sep = '|', encoding='latin-1', low_memory = False)
df_turma_sp = df_turma.loc[df_turma['CO_MUNICIPIO']==3550308,:]
df_turma_sp.dropna(subset = ['TP_ETAPA_ENSINO'], inplace = True)
df_turma_sp.loc[:,'TP_ETAPA_ENSINO'] = df_turma_sp.loc[:,'TP_ETAPA_ENSINO'].astype(int)
df_turma_sp = pd.merge(df_turma_sp,df_etapas, left_on='TP_ETAPA_ENSINO', right_on='cod_etapa', how = 'left')

pivot_turma_sp = df_turma_sp.groupby(['CO_ENTIDADE', 'TP_DEPENDENCIA','grupo_etapa']).agg(turmas = ('QT_MATRICULAS','count'), alunos=('QT_MATRICULAS','sum')).reset_index()

pivot_turma_sp = pivot_turma_sp.replace({'TP_DEPENDENCIA':dict_dependencia})
pivot_turma_sp = pivot_turma_sp.merge(df_censo_SP.loc[:,['CO_ENTIDADE', 'NO_ENTIDADE']].drop_duplicates(), on='CO_ENTIDADE', how = 'left')
pivot_turma_sp = pivot_turma_sp.merge(df_lat_lng, left_on='CO_ENTIDADE',right_on= 'codesc', how = 'left')

pivot_turma_sp_valid = pivot_turma_sp.dropna()


In [15]:
#Gerar Mapas
for grupo in pivot_turma_sp_valid['grupo_etapa'].unique():

    plot_data = pivot_turma_sp_valid[pivot_turma_sp_valid['grupo_etapa']==grupo].reset_index()


    m = folium.Map(location=[-23.512199, -46.749])

    for _, r in distritos.iterrows():
        # Without simplifying the representation of each borough,
        # the map might not be displayed
        sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
        geo_j = sim_geo.to_json()
        geo_j = folium.GeoJson(data=geo_j,
                               style_function=lambda x: {'fillColor': 'orange'})
        folium.Popup(r['Nome']).add_to(geo_j)
        geo_j.add_to(m)


    for i in range(plot_data.shape[0]):
        coordinates = [plot_data.LAT[i],plot_data.LNG[i]]
            #assign a color marker for the type of volcano, Strato being the most common
        if plot_data.TP_DEPENDENCIA[i] == "Federal":
            type_color = "green"
        elif plot_data.TP_DEPENDENCIA[i] == "Municipal":
            type_color = "blue"
        elif plot_data.TP_DEPENDENCIA[i] == "Estadual":
            type_color = "red"
        else:
            type_color = "purple"

        popup = "Escola: " + str(plot_data.NO_ENTIDADE[i]) + '<br>' + \
                "Dependencia: " + str(plot_data.TP_DEPENDENCIA[i]) + '<br>' + \
                "Turmas: " + str(plot_data.turmas[i]) + '<br>' + \
                "Alunos: " + str(plot_data.alunos[i]) 
        radius = float(plot_data.alunos[i])
        m.add_child(folium.Circle(location = coordinates,radius = radius, fill = True, popup = popup, color = type_color, fill_color = type_color))



    macro = MacroElement()
    macro._template = Template(template)

    m.get_root().add_child(macro)

    title = 'Escolas Cidade São Paulo - Ensino '+grupo
    title_html = '''
                 <h3 align="center" style="font-size:16px"><b>{}</b></h3>
                 '''.format(title)   
    m.get_root().html.add_child(folium.Element(title_html))
    m.save(grupo+'.html')